# Analyzing Exergame SESSION 1 SpecParam Model
Now that poorly fitted models have been removed and our final set of spectra have been saved as a pandas dataframe in `../results/specparam/final_model/df_final.pkl` we should be able to load it in and analyze all spectra

## Analysis Plan
Throughout the notebook, we complete the following tasks:
1. Load the pickle file and fit group spectra
2. Create a .csv export for the following metrics:
    - All peak metrics (4 columns: cfs, pws, bws, model index)
        - Make sure you assign frequency bands based off of cfs value
    - All Model Metrics
        - Aperiodic parameters (2 columns: offsets, exps [in order models])
        - Errors
        - R-squared
    - **Make sure you use the model index to pull relevant information from `df_final`**
3. Plot mean spectra with standard deviation for all 11 clusters.
    - Plot with aperiodic fit as dotted line
    - Plot as flattened spectrum

In [1]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, colors, colorbar
import pandas as pd
import pickle

# Spectral parameterization imports
from specparam import SpectralGroupModel
from specparam.plts.aperiodic import plot_aperiodic_params, plot_aperiodic_fits

# Import custom function
from create_specparam_plots import *

### Fit Model and Spectra

In [2]:
df = pd.read_pickle('../results/specparam/final_model/df_final.pkl')
# Extract spectra from final dataframe
spectra = np.array([spec for spec in df['spectra']])
freqs = np.arange(251)
# Initialize and fit new SpectralGroupModel on cleaned data
fg = SpectralGroupModel(peak_width_limits=[2, 8], min_peak_height=0.2, peak_threshold=2,
                               max_n_peaks=6, verbose=False)
freq_range = [1, 55]
fg.fit(freqs, spectra, freq_range)
fg.print_results()

                                                                                                  
                                          GROUP RESULTS                                           
                                                                                                  
                            Number of power spectra in the Group: 5826                            
                                                                                                  
                        The model was run on the frequency range 1 - 55 Hz                        
                                 Frequency Resolution is 1.00 Hz                                  
                                                                                                  
                              Power spectra were fit without a knee.                              
                                                                                                  
          

### Export Summary Data as .csv files
They will be analyzed further to summarize our data for the manuscript

#### Extracting Parameters

In [3]:
# Extract peak parameters
peaks = fg.get_params('peak_params')  # 4 columns: cfs, pws, bws, model index
aps = fg.get_params('aperiodic_params')  # 2 columns: offsets, exps [in order of channels/models]
# Extract goodness-of-fit metrics
errors = fg.get_params('error')
r2s = fg.get_params('r_squared')

##### Peaks Export
For each peak that we export, we will need to export its model's following values from `df`:
1. subject
2. session
3. experience
4. component
5. cluster

In [4]:
peaks_export = pd.DataFrame(peaks, columns=['cf', 'pw', 'bw', 'model_ind'])

# Convert Model Indices to integers to index from df and MERGE
peaks_export['model_ind'] = peaks_export['model_ind'].astype(int)
peaks_merged = peaks_export.merge(
    df[['subject', 'session', 'experience', 'component', 'cluster']],
    left_on='model_ind',
    right_index=True,
    how='left'
)
print("Method 1 - Using pandas merge:")
print(peaks_merged.head())

# Creating a column for frequency bands
def assign_freq_band(cf_value):
    if 1 <= cf_value < 4:
        return "delta"
    elif 4 <= cf_value < 8:
        return "theta"
    elif 8 <= cf_value < 12:
        return "alpha"
    elif 12 <= cf_value < 20:
        return "low_beta"
    elif 20 <= cf_value < 30:
        return "high_beta"
    elif 30 <= cf_value < 55:
        return "gamma"
    else:
        return None  # Default if cf doesn't fall within the specified ranges
peaks_merged['freq_band'] = peaks_merged['cf'].apply(assign_freq_band)

# Save as .csv file for analysis on R
peaks_merged.to_csv('../results/specparam/final_model/peaks.csv', index=False)

Method 1 - Using pandas merge:
          cf        pw        bw  model_ind  subject session     experience  \
0  19.673625  0.382657  8.000000          0  exgm002      s1  digitbackward   
1  10.512218  0.267552  3.274746          1  exgm002      s1   digitforward   
2  19.360175  0.496107  8.000000          1  exgm002      s1   digitforward   
3  10.031609  0.710193  4.141904          2  exgm002      s1         gonogo   
4  18.723401  0.670072  8.000000          2  exgm002      s1         gonogo   

   component  cluster  
0          5       10  
1          5       10  
2          5       10  
3          5       10  
4          5       10  


##### Model Metrics Export
Just like above we will need to export each model's following values from `df`:
1. subject
2. session
3. experience
4. component
5. cluster

In [5]:
# Ensure all arrays are 2D
r2s = r2s.reshape(-1, 1)  # Reshape r2s to (12, 1)
errors = errors.reshape(-1, 1)  # Reshape errors to (12, 1)

# Horizontally stack aperiodic params, R^2, and errors
model_metrics = pd.DataFrame(
    data=np.hstack([aps, r2s, errors]),  # Horizontally stack arrays
    columns=["offset", "exp", "r2", "error"]  # Column names
)

# Add Model Index for set up for joining with model information
model_metrics["model_ind"] = np.arange(len(df), dtype=int)

# Add Model Details to model_metric dataframe
model_metrics_merged = model_metrics.merge(
    df[['subject', 'session', 'experience', 'component', 'cluster']],
    left_on='model_ind',
    right_index=True,
    how='left'
)

# Write .csv to results folder
model_metrics_merged.to_csv('../results/specparam/final_model/model_metrics.csv', index=False)

## Creating Plots for each cluster

In [6]:
create_all_plots(df, fg) # make sure you edit this function by the time you are analyzing two sessions!

Created plots for Cluster 10, Session s1
Created cognitive medication comparison plots for Cluster 10, Session s1
Created balance medication comparison plots for Cluster 10, Session s1
Created plots for Cluster 11, Session s1
Created cognitive medication comparison plots for Cluster 11, Session s1
Created balance medication comparison plots for Cluster 11, Session s1
Created plots for Cluster 12, Session s1
Created cognitive medication comparison plots for Cluster 12, Session s1
Created balance medication comparison plots for Cluster 12, Session s1
Created plots for Cluster 13, Session s1
Created cognitive medication comparison plots for Cluster 13, Session s1
Created balance medication comparison plots for Cluster 13, Session s1
Created plots for Cluster 3, Session s1
Created cognitive medication comparison plots for Cluster 3, Session s1
Created balance medication comparison plots for Cluster 3, Session s1
Created plots for Cluster 4, Session s1
Created cognitive medication compariso